# Registration Notebook

In [ ]:
from organoid import preprocessing
from organoid import reconstruction
import numpy as np
import tifffile
from pathlib import Path

If necessary, you can plot here the positions of all samples (from the metadata), to match the two sides

In [ ]:
path = ...
reconstruction.plot_positions(
    path_bottom_positions=Path(path) / rf"multipoints_bottom.xml",
    path_top_positions=Path(path) / f"multipoints_top.xml",
)

Give the path to your data and name of images

In [ ]:
path = ...
list_ref = ["1_bottom"]
list_float = ["1_top"]
channels = [
    "dapi",
    "ecad",
]  # example of channels. If you have only one channel, just put one element in the list

Create the folder structure necessary for the registration (separates the channels)

In [ ]:
reconstruction.create_folders(
    path, list_ref=list_ref, list_float=list_float, channels=channels
)

Register automatically

To register your floating image onto the reference one, you should have an idea of the transformation to apply.  From this approximative initial transformation, the algorithm will find the exact transformation to match the 2 sides.

If your image has multiple channels, one will be the reference one, registered first. The second part of the code executes the registration for the other channels, using the same transformation as computed for the reference.

In [ ]:
# from now on, we consider only one sample. If you have multiple samples, you can loop : list_ref[i]
filename_ref = list_ref[0]
filename_float = list_float[0]

channel_reference = "dapi"

# register the reference channel first
reconstruction.register(
    path_data=Path(path) / filename_ref / "raw",
    path_transformation=Path(path) / filename_ref / "trsf",
    path_registered_data=Path(path) / filename_ref / "registered",
    reference_image=f"{filename_ref}_{channel_reference}.tif",
    floating_image=f"{filename_float}_{channel_reference}.tif",
    input_voxel=[1, 1, 1],
    output_voxel=[1, 1, 1],
    compute_trsf=1,
    # example of transformation if the sample has been flipped between the 2 views.
    # trans1 is a translation before the rotation, trans2 is a translation after the rotation.
    trans1=[-10, 50, 0],  # XYZ
    rot=[180, 0, 0],
    test_init=0,
    trsf_type="rigid",
    depth=3,
    bbox=1,
    save_json=Path(path) / filename_ref,
)


# registering other channel using the transformation computed above
for channel in channels[
    1:
]:  # (we skip the reference channel that we assumed is the first)
    print(channel)
    reconstruction.register(
        path_data=Path(path) / filename_ref / "raw",
        path_transformation=Path(path) / filename_ref / "trsf",
        path_registered_data=Path(path) / filename_ref / "registered",
        reference_image=f"{filename_ref}_{channel}.tif",
        floating_image=f"{filename_float}_{channel}.tif",
        input_voxel=[1, 1, 1],
        output_voxel=[1, 1, 1],
        compute_trsf=0,
    )

Napari visualization (you need to have napari installed)

In [ ]:
filename_ref = list_ref[0]
filename_float = list_float[0]
channel = channels[1]
scale = (1, 1, 1)
reconstruction.check_napari(
    folder=Path(path) / f"{filename_ref}",
    reference_image=f"{filename_ref}_{channel}.tif",
    floating_image=f"{filename_float}_{channel}.tif",
    scale=scale,
)

Fuse the 2 registered sides into one array

In [ ]:
filename_ref = list_ref[0]
filename_float = list_float[0]
for ch in channels:
    image = reconstruction.fuse_sides(
        path_registered_data=Path(path) / filename_ref / "registered",
        reference_image_reg=f"{filename_ref}_{ch}.tif",
        floating_image_reg=f"{filename_float}_{ch}.tif",
        folder_output=Path(path) / filename_ref / "fused",
        name_output=rf"fuseddata_{ch}.tif",
        slope_coeff=10,  # slope of the weight profile : 5 corresponds to a low slope, wide fusion width and 25 to a strong slope, very thin fusion width.
    )

Merge all the channels in one multichannel image

In [ ]:
# the images should be named 'sampleid_channel.tif', eg with sampleid = fuseddata and channel = dapi : fuseddata_dapi.tif
reconstruction.write_hyperstacks(
    path=Path(path) / filename_ref / "fused",
    sample_id="fuseddata",
    dtype=np.uint16, #you can change the dtype if needed
    channels=channels
)

### Optional : Registration using landmarks

If the automatic registration is not satisfying, one option is to give more precise initial transformations to the algorithm.

For that, you need to define landmarks, using for example the Napari plugin. One landmark is a feature that you recognize in both the reference image and the floating image, that you will need to pinpoint with a marker of given label, this label has to be an integer that has the same value in both image.

Once you have at least 3 annotated landmarks (=3 labels in each image), give these landmarks to the manual_registration function, that will compute the transformation necessary to align them, and give this transformation as a starting point to the algorithm.


1) If you used the napari plugin and saved you json file 


In [ ]:
path_to_initial_transformation = ... #finishes with /initial_transformation.json

rotation, translation = reconstruction.transformation_from_plugin(path_to_initial_transformation)
print('rotation :',rotation, '\n translation :',translation)
reconstruction.register(
    path_data=Path(path) / filename_ref / "raw",
    path_transformation=Path(path) / filename_ref / "trsf",
    path_registered_data=Path(path) / filename_ref / "registered",
    reference_image=f"{filename_ref}_{channel}.tif",
    floating_image=f"{filename_float}_{channel}.tif",
    input_voxel=[
        1,
        1,
        1,
    ],  # if you use landmarks, better use the same voxel size in and out
    output_voxel=[1, 1, 1],
    compute_trsf=1,
    rot=rotation,
    trans2=translation,
    test_init=0,
    trsf_type="rigid",
    depth=3,
    bbox=0,
    save_json="",
)

#then you can check on Napari what the result looks like. If is is satisfying, in cell #5 you can adjust the initial transformation (rot and trans2

2) If you have did not use the Napari plugin, and have 2 images corresponding to landmarks for the reference and floating images

In [ ]:
filename_ref = list_ref[0]
filename_float = list_float[0]
scale = (1, 1, 1)
path_to_landmarks = ...
reference_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"reference_landmarks.tif"
)
floating_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"floating_landmarks.tif"
)
reference_landmarks = reference_landmarks.astype(np.uint16)
floating_landmarks = floating_landmarks.astype(np.uint16)
channel = "dapi"

rot, trans1, trans2 = reconstruction.manual_registration_fct(
    reference_landmarks=reference_landmarks,
    floating_landmarks=floating_landmarks,
    scale=(1, 1, 1),
)


reconstruction.register(
    path_data=Path(path_to_landmarks) / filename_ref / "raw",
    path_transformation=Path(path_to_landmarks) / filename_ref / "trsf",
    path_registered_data=Path(path_to_landmarks) / filename_ref / "registered",
    path_to_bin=path_to_bin,
    reference_image=f"{filename_ref}_{channel}.tif",
    floating_image=f"{filename_float}_{channel}.tif",
    input_voxel=[
        1,
        1,
        1,
    ],  # if you use landmarks, better use the same voxel size in and out
    output_voxel=[1, 1, 1],
    compute_trsf=1,
    rot=rot,
    trans1=trans1,
    trans2=trans2,
    test_init=0,
    trsf_type="rigid",
    depth=3,
    bbox=0,
    save_json="",
)

Debug : register the landmarks images and plot the result to see if the transformation is correct

In [ ]:
path_to_landmarks = ...

init_trsfs = [["rot", "Z", 180]]
reconstruction.register(
    path_data=Path(path_to_landmarks) / filename_ref / "raw",
    path_transformation=Path(path_to_landmarks) / filename_ref / "trsf",
    path_registered_data=Path(path_to_landmarks) / filename_ref / "registered",
    reference_image=f"reference_landmarks.tif",  # they have to be in 16bit to be registered
    floating_image=f"floating_landmarks.tif",
    input_voxel=[1, 1, 1],
    output_voxel=[1, 1, 1],
    compute_trsf=1,
    other_trsf=init_trsfs,
    test_init=1,  # keep test_init=1 to see only the provided tranformation
    trsf_type="rigid",
    depth=3,
    save_json=Path(path_to_landmarks) / filename_ref,
)

# loading the output and add articially the center of mass of the points, to see how well they align

reference_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"registered/reference_landmarks.tif"
)
floating_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"registered/floating_landmarks.tif"
)
reference_landmarks = reconstruction.add_centermass(reference_landmarks)
floating_landmarks = reconstruction.add_centermass(floating_landmarks)

# loading the raw landmarks, before the registration
reference_landmarks_notreg = tifffile.imread(
    Path(path_to_landmarks) / f"raw/reference_landmarks.tif"
)
floating_landmarks_notreg = tifffile.imread(
    Path(path_to_landmarks) / f"raw/floating_landmarks.tif"
)
reference_landmarks_notreg = reconstruction.add_centermass(
    reference_landmarks_notreg
)
floating_landmarks_notreg = reconstruction.add_centermass(
    floating_landmarks_notreg
)

# plotting the center of the image
image_center = np.zeros_like(reference_landmarks)
z, y, x = np.array(reference_landmarks.shape) / 2
image_center[
    int(z) - 10 : int(z) + 10,
    int(y) - 10 : int(y) + 10,
    int(x) - 10 : int(x) + 10,
] = 100

reconstruction.check_napari(
    folder=rf"{path}\{filename_ref}",
    reference_image=reference_landmarks,
    floating_image=floating_landmarks,
    additional_images=[
        reference_landmarks_notreg,
        floating_landmarks_notreg,
        image_center,
    ],
    names_additional_images=[
        "reference_landmarks_notreg",
        "floating_landmarks_notreg",
        "image_center",
    ],
    scale=scale,
    labels=True,
)